In [1]:
import pandas as pd
from poem_env import PoemGenerationEnv
from poem_dql import DQLAgent

2024-12-28 09:17:57.553847: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


done


In [13]:
df = pd.read_csv('../data/temp_romanian_words.csv')
df = df[df['joint_syllables'] != 'not-found']
df.to_csv('../data/temporary_romanian_words.csv', index=False)

In [14]:
# Clean data
df = pd.read_csv('../data/temporary_romanian_words.csv')
syls = list(df['syllables'])
new_syls = []
for s in syls:
    s = eval(s)
    i=0
    while i < (len(s)):
        if '(' in s[i]:
            break
        i+=1
    new_syls.append(s[:i])
            
print (len(new_syls), len(df))
df['syllables'] = new_syls
df.to_csv('../data/temporary2_romanian_words.csv', index=False)

12719 12719


In [15]:
df = pd.read_csv('../data/temporary2_romanian_words.csv')

In [17]:
# Initialize the environment
env = PoemGenerationEnv(df, num_verses=2, syllables_per_verse=6, rhyme_data=3)

# Interact with the environment
state = env.reset()
done = False

while not done:
    action = env.sample_action()  # Random action (replace with DQL model's action)
    state, reward, done = env.step(action)
    print(f"State: {state}")
    print(f"Reward: {reward}")
    print(f"Done: {done}")

here2
here3
here4
here5
State: {'poem': [], 'verse': ['bătuci'], 'syllables_left': 3, 'verse_index': 0}
Reward: 1
Done: False
State: {'poem': [], 'verse': ['bătuci', 'autocontempla'], 'syllables_left': -3, 'verse_index': 0}
Reward: -15
Done: True


In [ ]:
# Hyperparameters
episodes = 1000
batch_size = 32
state_size = 3  # For example: poem length, syllables left, verse index
action_size = len(vocab)  # Vocabulary size
target_update_freq = 10

In [ ]:


# Initialize the environment and agent
env = PoemGenerationEnv(vocab, syllable_data, rhyme_data, num_verses=2, syllables_per_verse=6)
agent = DQLAgent(state_size, action_size)

for episode in range(episodes):
    # Reset environment at the start of each episode
    state = env.reset()
    state_vector = np.array([state["verse_index"], state["syllables_left"], len(state["poem"])])
    total_reward = 0

    done = False
    while not done:
        # Choose an action
        action = agent.act(state_vector)

        # Take the action in the environment
        next_state, reward, done = env.step(action)
        next_state_vector = np.array([next_state["verse_index"], next_state["syllables_left"], len(next_state["poem"])])

        # Store experience
        agent.remember(state_vector, action, reward, next_state_vector, done)

        # Update state
        state_vector = next_state_vector
        total_reward += reward

        # Replay and train
        agent.replay(batch_size)

    # Update target network periodically
    if episode % target_update_freq == 0:
        agent.update_target_network()

    print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2f}")
